In [1]:
# Install dependencies
!pip install langchain chromadb sentence-transformers
!pip install langchain-community
!pip install PyPDF2

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load a pre-trained embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-5-701a5f82b8cb>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

In [11]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from PyPDF2 import PdfReader

# Load documents from files
def load_documents_from_files(file_paths):
    docs = []
    for file_path in file_paths:
        if file_path.endswith(".txt"):
            with open(file_path, "r", encoding="utf-8") as f:
                docs.append(f.read())
        elif file_path.endswith(".pdf"):
            reader = PdfReader(file_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            docs.append(text)
        else:
            print(f"Unsupported file type: {file_path}")
    return docs

# Example file paths (replace with your actual files)
file_paths = ["/content/test.txt", "/content/ASSIGNMENT 02.pdf"]
docs = load_documents_from_files(file_paths)

# Split documents into smaller chunks for better indexing
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.create_documents(docs)

# Set up ChromaDB
vectorstore = Chroma.from_documents(documents=chunks, embedding=embedding_model)


In [12]:
query = "What services does Verizon offer?"

# Perform a similarity search
results = vectorstore.similarity_search(query, k=5)

# Retrieve the context
context = [result.page_content for result in results]
print("Retrieved Context:", context)


Retrieved Context: ['Verizon offers a wide range of wireless plans and internet services.', 'Verizon is a telecommunications company providing 5G services.', 'T-Mobile is another major telecom provider in the USA.', 'ASSIGNMENT 02  \n \n \n \n \n \n \n \n \n \n \n \n \n \nTeam Members,   \nThida s,  \nDulmi ,  \nShane    Telecommuni cations  \n \nVerizon is one of the biggest telecommunications companies in \nthe United States. Verizon has previously sponsored a variety of \nevents, sports teams, and community programmers . \nVerizon\'s participation in sports partnerships is a substantial \nsponsorship. They\'ve had substantial sponsorships in both the \nNational Football League (NFL) and the National Basketball \nAssociation. Verizon\'s NFL sponsorship has included efforts like \nthe "Verizon Up" program s, which provides rewards and \nexperiences to Verizon consumers . \n \nSportswear  \nNike is an American  athletic footwear and apparel corpor ation . \nHeadquarte red near Beaverto

In [8]:
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959643 sha256=2059afdd2dbe54b3ef9479606a6343d63d277c0b643cc1a6ae4fe0229bce043c
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [ ]:
from langchain.llms import LlamaCpp

# Path to your Llama model
llama_model_path = "./llama-7b.ggmlv3.q4_0.bin"

# Load the Llama model
llm = LlamaCpp(model_path=llama_model_path)


In [ ]:
from langchain.chains import RetrievalQA

# Create a retriever
retriever = vectorstore.as_retriever()

# Create the RAG pipeline with Llama
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Ask a question
query = "What services does Verizon offer?"
response = qa_chain.run(query)
print("Response:", response)